In [ ]:
import torch
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from skimage import measure
from skimage.transform import resize
import plotly.graph_objects as go
import time
import os
import gc
device = 'cuda:0'
torch.manual_seed(0)
torch.cuda.is_available()

In [ ]:
Molecule = {
    "name": "Water",
    "element": [
        8,
        1,
        1
    ],
    "x": [
        0,
        0.7566 * np.sqrt(1/2),
        -0.7566 * np.sqrt(1/2)
    ],
    "y": [
        0,
        -0.7566 * np.sqrt(1/2),
        0.7566 * np.sqrt(1/2)
    ],
    "z": [
        0,
        -0.5942,
        -0.5942
    ]
}

In [ ]:
Config = {
    "dx": 0.3,
    "N": [100, 100, 100],
    "L": [30, 30, 30],
    "Z": 0,
    "lr": 0.1,
    "fast": False,
    "lumo": 0,
    "ion_ene": [0, 13.59844, 24.58738, 
                5.39171, 9.32269, 8.29803, 11.26030, 14.53414, 13.61806, 17.42282, 21.5646,
                5.13908, 7.64624, 5.98577, 8.15169, 10.48669, 10.36001, 12.96764, 15.75962]
}
Train = {
    "N": None,
    "density_i": [],
    "density_o": [],
    "density_f": [],
    "difference": []
}
Result = {
    "orbits": None,
    "orbits_e": None,
    "density": None,
    "energy": None,
    "dipole": None,
    "charge": None,
    "shape": None,
    "space": None,
    "grid": None,
    "Q_atom": None,
    "N_atom": None,
    "loss": None
}

In [ ]:
# Visualize Function
def visualize(func, thres=1e-6):
    verts, faces, _, _ = measure.marching_cubes(func, thres, spacing=(0.1, 0.1, 0.1))
    intensity = np.linalg.norm(verts, axis=1)

    fig = go.Figure(data=[go.Mesh3d(x=verts[:, 0], y=verts[:, 1], z=verts[:, 2],
                                    i=faces[:, 0], j=faces[:, 1], k=faces[:, 2],
                                    intensity=intensity,
                                    colorscale='Agsunset',
                                    opacity=1.0)])

    fig.update_layout(scene=dict(xaxis=dict(visible=False),
                                 yaxis=dict(visible=False),
                                 zaxis=dict(visible=False),
                                 bgcolor='rgb(0, 0, 0)'),
                      margin=dict(l=0, r=0, b=0, t=0))
    fig.show()

In [ ]:
# Density Functional Theory
def calculate(config, molecule, train, result):
    t_start = time.time()
    gc.collect()
    torch.cuda.empty_cache()
    
    # Config
    Atom = []
    Qm = np.zeros((3,3))
    for i in range(len(molecule["element"])):
        Atom.append([molecule["element"][i], np.array([molecule["x"][i], molecule["y"][i], molecule["z"][i]]) * 1.889726125])
        Qm[:,0] = np.maximum(Qm[:,0], Atom[i][1])
        Qm[:,1] = np.minimum(Qm[:,1], Atom[i][1])
        Qm[:,2] += Atom[i][0] * Atom[i][1]
    Qc = Qm[:,2] / sum(molecule["element"])
    Qm -= Qc
    for a in Atom:
        a[1] -= Qc
    dx = config["dx"]
    N = config["N"]
    L = config["L"]
    Z = config["Z"] + sum(molecule["element"])
    lr = config["lr"]
    fast = config["fast"]
    lumo = config["lumo"]
    ion_ene = config["ion_ene"]
    
    # Grid Space
    Q = np.zeros((3, N[0], N[1], N[2]))
    Q[0,:,:,:] = np.linspace(-L[0]/2, L[0]/2, N[0])[:, np.newaxis, np.newaxis]
    Q[1,:,:,:] = np.linspace(-L[1]/2, L[1]/2, N[1])[np.newaxis, :, np.newaxis]
    Q[2,:,:,:] = np.linspace(-L[2]/2, L[2]/2, N[2])[np.newaxis, np.newaxis, :]
    
    # Initial Density
    if len(train["difference"]) == 0:
        NI = 0
        for a in Atom:
            Za = a[0]
            Qa = a[1][:, np.newaxis, np.newaxis, np.newaxis]
            Ra = np.sqrt(np.sum((Q-Qa)*(Q-Qa), axis=0))
            Ia = ion_ene[Za] / 27.211
            NI += (Za*np.sqrt(512*(Ia**3))) / (8*np.pi) * np.exp(-np.sqrt(8*Ia)*Ra)
        NI = NI.reshape(N[0]*N[1]*N[2])
        NI *= Z / np.sum(NI*(dx**3))
        train["density_i"] = [NI, NI, NI]
        train["density_o"] = [NI, NI, NI]
        train["density_f"] = [0, 0, 0]
        train["N"] = N
        result["loss"] = 1e9
        print("L:", L, " N:", N, " dx", dx)
        print("Initial:")
        visualize(NI.reshape(N), 1e-1)
    
    # Density Mixing
    NIp = train["density_i"]
    NOp = train["density_o"]
    NFp = train["density_f"]
    pX = np.array([1, 0, 0])
    if len(train["difference"])>3 and train["difference"][-3]<sum(molecule["element"])*0.3:
        pA = np.array([[np.sum(NFp[0]*NFp[0]), np.sum(NFp[0]*NFp[1]), np.sum(NFp[0]*NFp[2]), 1],
                       [np.sum(NFp[1]*NFp[0]), np.sum(NFp[1]*NFp[1]), np.sum(NFp[1]*NFp[2]), 1],
                       [np.sum(NFp[2]*NFp[0]), np.sum(NFp[2]*NFp[1]), np.sum(NFp[2]*NFp[2]), 1],
                       [                    1,                     1,                     1, 0]])
        pB = np.array([0, 0, 0, 1])
        pX = np.linalg.solve(pA, pB)
    NIm = pX[0] * NIp[0] + pX[1] * NIp[1] + pX[2] * NIp[2]
    NOm = pX[0] * NOp[0] + pX[1] * NOp[1] + pX[2] * NOp[2]
    NI = (1-lr) * NIm + lr * NOm
    if np.min(NI) < 0:
        NI = (1-lr) * NIp[0] + lr * NOp[0]
    NI *= Z / np.sum(NI*(dx**3))
    
    # Kinetic Energy
    D = [sp.sparse.spdiags(np.array([np.ones([N[i]]), -2*np.ones([N[i]]), np.ones([N[i]])]), 
                           np.array([-1,0,1]), N[i], N[i]) 
         for i in range(3)]
    Lap = sp.sparse.kronsum(sp.sparse.kronsum(D[2],D[1]), D[0]) / (dx**2)
    T = -1/2 * Lap

    # External Energy
    V_ext = 0
    for a in Atom:
        Za = a[0]
        Qa = a[1][:, np.newaxis, np.newaxis, np.newaxis]
        V_ext += -Za / (np.sqrt(np.sum((Q-Qa)*(Q-Qa), axis=0)) + 1e-6)
    V_ext = sp.sparse.diags(V_ext.reshape(N[0]*N[1]*N[2]))
    
    # Hartree Energy
    V_har0 = sp.sparse.linalg.cg(Lap, -4*np.pi*NI)[0]
    V_har = sp.sparse.diags(V_har0)
    
    # Exchange Energy
    rho = torch.tensor(NI.reshape(N), requires_grad=True)
    g_rho = torch.gradient(rho)
    g_rho = torch.sqrt(g_rho[0]*g_rho[0] + g_rho[1]*g_rho[1] + g_rho[2]*g_rho[2]) / dx
    ep_x = -(3/4) * np.power(3/np.pi, 1/3) * torch.pow(rho, 1/3)
    px = (g_rho/torch.pow(rho, 4/3)) * (2/9) * np.power(np.pi/3, 1/3)
    ED_x = rho * ep_x * (3*(px**2)+(np.pi**2)*torch.log(px+1)) / ((3*px+np.pi**2)*torch.log(px+1))
    ES_x = torch.sum(ED_x)
    ES_x.backward()
    V_exc0 = rho.grad.detach().numpy().reshape(N[0]*N[1]*N[2])
    V_exc = sp.sparse.diags(V_exc0)
    
    # Correlation Energy
    rho = torch.tensor(NI.reshape(N), requires_grad=True)
    g_rho = torch.gradient(rho)
    g_rho = torch.sqrt(g_rho[0]*g_rho[0] + g_rho[1]*g_rho[1] + g_rho[2]*g_rho[2]) / dx
    pa = (np.log(2)-1) / (2*(np.pi**2))
    pb = 20.4562557
    rs = torch.pow(4*np.pi*rho/3, -1/3)
    ep_c = pa * torch.log(1+pb/rs+pb/(rs**2))
    pt = np.power(np.pi/3, 1/6) * (1/4) * (g_rho/torch.pow(rho, 7/6))
    ph = 0.06672632
    ED_c = rho * ep_c * torch.pow(1+pt**2, ph/ep_c)
    ES_c = torch.sum(ED_c)
    ES_c.backward()
    V_cor0 = rho.grad.detach().numpy().reshape(N[0]*N[1]*N[2])
    V_cor = sp.sparse.diags(V_cor0)
    
    # Solve
    H = (T + V_ext + V_har + V_exc + V_cor).tocoo()
    H = torch.sparse_coo_tensor(indices=torch.tensor(np.vstack([H.row, H.col])), values=torch.tensor(H.data), size=H.shape).to(device)
    if fast:
        H = H.float()
    fn = [2 for i in range(Z//2)]
    if Z % 2 == 1:
        fn.append(1)
    eigval, eigvec = torch.lobpcg(H, len(fn)+lumo, largest=False)

    # Density
    orbits_e = eigval.detach().cpu().numpy()
    orbits = eigvec.T.detach().cpu().numpy()
    orbits = orbits / np.sqrt(np.sum(orbits*orbits*(dx**3), axis=1))[:, np.newaxis]
    NO = np.zeros(N[0]*N[1]*N[2], dtype=np.float32)
    for ne, orb in zip(fn, orbits[:len(fn)]):
        NO += ne * (orb**2)
    NO *= Z / np.sum(NO*(dx**3))
    NF = NO - NI
    Dif = np.sum(np.abs(NF) * (dx**3))
    
    # Distribution
    QA = np.array([a[1] for a in Atom], dtype='float64').T
    NA = np.array([a[0] for a in Atom], dtype='float64')
    QE = Q.reshape((3, N[0]*N[1]*N[2]))
    NE = NI * (dx**3)

    # Total Energy
    EN = 0
    for ne, orb_e in zip(fn, orbits_e[:len(fn)]):
        EN += ne * orb_e
    EN -= np.sum((1/2) * V_har0 * NI * (dx**3))
    EN -= np.sum(V_exc0 * NI * (dx**3))
    EN -= np.sum(V_cor0 * NI * (dx**3))
    EN += np.sum(ED_x.detach().numpy() * (dx**3))
    EN += np.sum(ED_c.detach().numpy() * (dx**3))
    for i in range(len(Atom)):
        for j in range(i+1, len(Atom)):
            EN += Atom[i][0] * Atom[j][0] / np.sqrt(np.sum((Atom[i][1]-Atom[j][1])**2))

    # Dipole Moment
    DM = np.zeros(3)
    DM += np.sum(NA[np.newaxis, :]*QA, axis=1)
    DM += np.sum(-NE[np.newaxis, :]*QE, axis=1)

    # Partial Charge
    PC = np.zeros(len(Atom))
    dis = np.zeros((N[0]*N[1]*N[2], len(Atom)))
    for i, a in enumerate(Atom):
        Qa = a[1][:, np.newaxis, np.newaxis, np.newaxis]
        dis[:,i] = np.sqrt(np.sum((Q-Qa)*(Q-Qa), axis=0)).reshape(N[0]*N[1]*N[2])
    arg = np.argmin(dis, axis=1)
    np.add.at(PC, arg, NE)
    PC = NA - PC
    
    # Train
    train["density_i"] = [NI, NIp[0], NIp[1]]
    train["density_o"] = [NO, NOp[0], NOp[1]]
    train["density_f"] = [NF, NFp[0], NFp[1]]
    train["difference"].append(Dif)
    
    # Result
    if Dif < result["loss"]:
        result["orbits"] = orbits
        result["orbits_e"] = orbits_e
        result["density"] = NI
        result["energy"] = EN
        result["dipole"] = DM
        result["charge"] = PC
        result["shape"] = N
        result["space"] = L
        result["grid"] = dx
        result["Q_atom"] = QA.T
        result["N_atom"] = NA
        result["loss"] = Dif
    
    t_end = time.time()
    print("Iteration:", len(train["difference"]), " / Time:", t_end-t_start)
    print("Difference:", Dif, "Energy:", EN)
    visualize(NO.reshape(N), 1e-1)

In [ ]:
prel = len(Train["difference"])
for i in range(prel, 100):
    calculate(Config, Molecule, Train, Result)    

In [ ]:
Result

In [ ]:
np.linalg.norm(Result["dipole"])

In [ ]:
visualize(Result["density"].reshape(Result["shape"]), 1e-1)

In [ ]:
for orb in Result["orbits"]:
    visualize((orb*orb*sum(Molecule["element"])).reshape(Result["shape"]), 1e-3)

In [ ]:
np.savez(Molecule["name"], **Result)

In [ ]:
a = np.array([1.2, 4.7, -4.9])
print(a.dtype)
np.round(a)

In [ ]:
Train["energy"] = []

In [ ]:
Train["energy"]

In [ ]:
Config["lumo"] = 3